In [1]:
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.ui import Select
import time

In [2]:
# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
        
    global web
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
        
    return web


In [3]:
# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
    
    from time import sleep 
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 700
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(4)
        
        try:
            specs = web.find_element(by="xpath", value="/html/body/div[1]/div/main/section/button").click()
        except Exception as error:
            pass

In [4]:
def search():
    input_search = input("inter search : ")
    location = input("inter your location 'defult is Egypt': ")
    
    search = input_search.split()
    if not location:
        location = "Egypt"
    
    if len(search) == 2:
        link = f"https://www.linkedin.com/jobs/search/?currentJobId=3778287041&geoId=106155005&keywords={search[0]}%20{search[-1]}&location={location}&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true"


    elif len(search) == 3:
        link = f"https://www.linkedin.com/jobs/search/?currentJobId=3788648090&geoId=106155005&keywords={search[0]}%20{search[1]}%20{search[-1]}&location={location}&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true"


    elif len(search) == 4:
        link = f"https://www.linkedin.com/jobs/search/?currentJobId=3780610148&geoId=106155005&keywords={search[0]}%{search[1]}%{search[2]}%20{search[-1]}&location={location}&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"

    else:
        link = f"https://www.linkedin.com/jobs/search/?currentJobId=3786926278&geoId=106155005&keywords={''.join(search)}&location={location}&origin=JOB_SEARCH_PAGE_LOCATION_SUGGESTION&refresh=true"


    print(location)
    return link, input_search, location

In [5]:
link, input_search, location = search()
web = open_chromedriver()

inter search : web development
inter your location 'defult is Egypt': 
Egypt


C:\Users\ammar\AppData\Local\Temp\ipykernel_14300\1965407970.py:18: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


In [6]:
def get_links(link=link):
    try:
        try:
            web.get(link) # open link
        except:
            time.sleep(2)
            web.get(link)
        scroll_down_all_the_way()
        time.sleep(3)
        
        soup = BeautifulSoup(web.page_source, 'html.parser') # send request

        item = soup.find("ul", {"class": "jobs-search__results-list"})
        links = [i.a["href"]for i in item.find_all("li")]

    
    except:
        time.sleep(3)
        
        soup = BeautifulSoup(web.page_source, 'html.parser') # send request
        item = soup.find("ul", {"class": "jobs-search__results-list"})
        links = [i.a["href"]for i in item.find_all("li")]
    
    return links

links = get_links()
print(len(links))

753


In [7]:
def get_job_info():
    listings = []
    link_len = len(links)
    for n, link in enumerate(links, start=1):
        try:
            web.get(link)

            time.sleep(2)
            try:
                specs = web.find_element(by="xpath", value="/html/body/main/section[1]/div/div/section[1]/div/div/section/button[1]").click()
            except:
                web.get(link)
                time.sleep(2)
                specs = web.find_element(by="xpath", value="/html/body/main/section[1]/div/div/section[1]/div/div/section/button[1]").click()
                
            soup = BeautifulSoup(web.page_source, 'html.parser') # send request

            listing = {}
            listing["job_link"] = link
            listing["job_name"] = soup.h1.text
            listing["label"] = input_search


            info = soup.find("h4", {"class": "top-card-layout__second-subline font-sans text-sm leading-open text-color-text-low-emphasis mt-0.5"})
            company_info = [i.get_text(strip=True) for i in info.find_all("span")]
            listing["company_name"] = company_info[0]
            listing["post_time"] = company_info[2]
            location = company_info[1].split(", ")
            listing["city_name"] = ", ".join(location[:-1])
            listing["country_name"] = location[-1]

            num_applicant = soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"})
            if num_applicant:
                listing["num_applicant"] = num_applicant.get_text(strip=True)

            description = soup.find("div", {"class": "decorated-job-posting__details"})

            head_liens = [i.get_text(strip=True) for i in description.find_all("strong")]
            if len(head_liens) > 2:
                listing["about_company"] = head_liens[0]

            job_diss = description.find_all("ul")        
            listing["job_responsibilities"] = job_diss[0].get_text(strip=True)

            job_req = []
            for i in [i.find_all("li") for i in job_diss[1:]]:
                for i in i:
                    job_req.append(i.get_text(strip=True))
            listing["job_requirements"] = " ; ".join(job_req)

            description_list = soup.find("ul", {"class": "description__job-criteria-list"})
            keys = [i.get_text(strip=True) for i in description_list.find_all("h3")]
            values = [i.get_text(strip=True) for i in description_list.find_all("span")]
            dict_value = dict(zip(keys, values))
            listing.update(dict_value)

            listings.append(listing)
            print(f"current job number is : {n} of : {link_len}")
            print("\n", "-"*100, listing, "\n")
        
        except Exception as error:
            print(f"Error is '\n'{error} \n")

    return listings

In [8]:
listings = get_job_info()

current job number is : 1 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/junior-frontend-developer-remote-internship-it-at-talentkompass-deutschland-3786035181?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=14zZZoxe2QuAEuYvNfyh6w%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_name': 'Junior Frontend Developer (Remote Internship - IT)', 'label': 'web development', 'company_name': 'TalentKompass Deutschland', 'post_time': '1 week ago', 'city_name': 'Luxor, Luxor', 'country_name': 'Egypt', 'num_applicant': '178 applicants', 'about_company': 'LOCATION:', 'job_responsibilities': 'Assist in the development, testing, and maintenance of web applications using HTML, CSS, and JavaScriptCollaborate with cross-functional teams to gather requirements and design user interfacesDebug and troubleshoot frontend issues, ensuring optimal performance and user experien

current job number is : 6 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/junior-software-developer-at-espace-3783251544?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=I%2FBxffzQNRCOzY4epzmhHg%3D%3D&position=6&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_name': 'Junior Software Developer', 'label': 'web development', 'company_name': 'eSpace', 'post_time': '2 weeks ago', 'city_name': 'Alexandria, Alexandria', 'country_name': 'Egypt', 'about_company': 'Job Description', 'job_responsibilities': 'Develop and implement new software programs.Producing efficient and elegant code based on requirementsMake good technical decisions that provide solutions to business challengesWrite Integration and Unit test for the written code.Troubleshooting, debugging, maintaining and improving existing software.Maintain and recommend software improvements to ensure strong functionality 

current job number is : 11 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/full-stack-developer-bootcamp-online-%E2%80%93-part-time-%E2%80%93-gain-your-first-hands-on-experience-at-moyyn-3794786115?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=XXV8fLfdn4axjwJf5vKOEQ%3D%3D&position=11&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_name': 'Full-stack Developer Bootcamp (online – part-time) – Gain your first hands-on experience', 'label': 'web development', 'company_name': 'Moyyn', 'post_time': '10 hours ago', 'city_name': 'Cairo, Cairo', 'country_name': 'Egypt', 'about_company': 'Program Overview', 'job_responsibilities': 'Hands-on Training and Project Work:', 'job_requirements': 'Full-stack Training: ; Career Guidance: ; Seniority levelEntry level ; Employment typePart-time ; Job functionEngineering and Information Technology ; IndustriesStaffing and Recruiting', 'S

current job number is : 17 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/front-end-developer-at-rekrut-by-talent-360-3743198668?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=NR9iJxp2XjcFBqJlzN6fQw%3D%3D&position=17&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_name': 'Front-End Developer', 'label': 'web development', 'company_name': 'Rekrut by Talent 360', 'post_time': '2 months ago', 'city_name': 'Cairo, Cairo', 'country_name': 'Egypt', 'job_responsibilities': 'BSc in Computer Science or Engineering2-3 years of experience as a Front-End Developer with proven experience in the following languages: HTML,CSS , javascript ,jQuery.Proven experience in Front-End libraries and framework (e.g. Vue.js, Reactjs, angular).Experience in reviewing codes & testing new features to ensure they are compatible with existing features [ Debugging ].Experience with SCRUM developmen

current job number is : 21 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/web-developer-ft-at-samtech-middle-east-fz-llc-3778208549?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=3g1sk2foxTVuMjlwca1Q8w%3D%3D&position=21&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_name': 'Web Developer (FT)', 'label': 'web development', 'company_name': 'SamTech Middle East Fz LLC', 'post_time': '3 weeks ago', 'city_name': 'Cairo, Cairo', 'country_name': 'Egypt', 'num_applicant': '49 applicants', 'about_company': 'Position:', 'job_responsibilities': 'Strong knowledge and development experience in PHP, CSS3/HTML5, JavaScript, jQuery, MySQL.Experience in WordPress development, administration, website content management.Ensuring high-performance, 24x7 availability of company’s corporate website and managing all technical aspects of the CMS.Ability to convert comprehensive layout and 

...

current job number is : 738 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/c-golang-software-engineer-working-on-dqlite-a-raft-extension-for-sqlite-at-canonical-3768271617?refId=zfYF0L7t%2B7zVn1ucw2Cv4Q%3D%3D&trackingId=jt2zPYs8SESCY6VGnHpCZQ%3D%3D&position=19&pageNum=29&trk=public_jobs_jserp-result_search-card', 'job_name': 'C, Golang Software Engineer working on dqlite, a Raft extension for SQLite', 'label': 'web development', 'company_name': 'Canonical', 'post_time': '1 month ago', 'city_name': 'Cairo, Cairo', 'country_name': 'Egypt', 'job_responsibilities': 'Design and implement features across dqliteDebug and fix issues encountered by your usersImprove Jepsen tests, traditional HA database automated testing and stress testsParticipate in our engineering process through code and architectural reviewEngage with the open source community and commercial partners', 'job_requ

current job number is : 741 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/staff-lead-software-engineer-fintech-bangkok-based-%E2%80%93-relocation-provided-at-agoda-3742049545?refId=zfYF0L7t%2B7zVn1ucw2Cv4Q%3D%3D&trackingId=8WuH5P%2B31KhaRNWke7yPgQ%3D%3D&position=22&pageNum=29&trk=public_jobs_jserp-result_search-card', 'job_name': 'Staff/ Lead Software Engineer (FinTech) [Bangkok based – Relocation provided]', 'label': 'web development', 'company_name': 'Agoda', 'post_time': '6 days ago', 'city_name': 'Giza, Al Jizah', 'country_name': 'Egypt', 'num_applicant': '30 applicants', 'about_company': 'About Agoda', 'job_responsibilities': 'Think and own the full life cycle of our products, not just a single piece of code – from business requirements, technology selection , coding standards, agile development, unit and application testing , to CI/CD and proper monitoringDesign, deve

current job number is : 745 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/lead-software-engineer-bangkok-based-%E2%80%93-relocation-provided-at-agoda-3714146402?refId=Scl1I%2BxPruRdLGmiDF%2FoGA%3D%3D&trackingId=1mRajXaNJtoJ4lQ%2FC0y0ZQ%3D%3D&position=1&pageNum=30&trk=public_jobs_jserp-result_search-card', 'job_name': 'Lead Software Engineer (Bangkok based – Relocation provided)', 'label': 'web development', 'company_name': 'Agoda', 'post_time': '1 day ago', 'city_name': 'Alexandria, Alexandria', 'country_name': 'Egypt', 'num_applicant': '53 applicants', 'about_company': 'About Agoda', 'job_responsibilities': '8+ years’ experience under your belt developing performance-critical applications that run in a production environment using Scala and JavaA minimum of one year of experience with ScalaYou have RDBMS knowledge (SQL Server, Oracle, MySql or other)You have a Good command

current job number is : 749 of : 753

 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://eg.linkedin.com/jobs/view/principal-is-bp-engineer-at-valeo-3687027884?refId=Scl1I%2BxPruRdLGmiDF%2FoGA%3D%3D&trackingId=BZ3MEfRmMSiDqJ67BY6Sww%3D%3D&position=5&pageNum=30&trk=public_jobs_jserp-result_search-card', 'job_name': 'Principal IS BP Engineer', 'label': 'web development', 'company_name': 'Valeo', 'post_time': '1 week ago', 'city_name': 'Cairo, Cairo', 'country_name': 'Egypt', 'num_applicant': '58 applicants', 'about_company': 'Work Complexity', 'job_responsibilities': 'Coordinates and works on multiple cross-functional base work initiatives and projects.Supports multiple systems or applications of medium to high complexity (complexity defined by size, technology used, and system feeds and interfaces) with multiple concurrent users, ensuring control, integrity, and accessibilityActs as the technical liaison between R&

Error is '
'Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/section[1]/div/div/section[1]/div/div/section/button[1]"}
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x009E6EE3+174339]
	(No symbol) [0x00910A51]
	(No symbol) [0x00626FF6]
	(No symbol) [0x00659876]
	(No symbol) [0x00659C2C]
	(No symbol) [0x0068BD42]
	(No symbol) [0x00677054]
	(No symbol) [0x0068A104]
	(No symbol) [0x00676DA6]
	(No symbol) [0x00651034]
	(No symbol) [0x00651F8D]
	GetHandleVerifier [0x00A84B1C+820540]
	sqlite3_dbdata_init [0x00B453EE+653550]
	sqlite3_dbdata_init [0x00B44E09+652041]
	sqlite3_dbdata_init [0x00B397CC+605388]
	sqlite3_dbdata_init [0x00B45D9B+656027]
	(No symbol) [0x0091FE6C]
	(No symbol) [0x009183B8]
	(No symbol) [0x009184DD]
	(No symbol) [0x00905818]
	BaseThreadInitThunk [0x7613FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x775C7C6E+286]
	RtlGetAppContainerNamedObjectPath [0x775C7C3E+238]
 

current job number is : 753 of

In [9]:
len(listings)

619

In [10]:
import pandas as pd
pd.options.display.max_columns = None # donot display all rows and columns of data

df = pd.DataFrame.from_records(listings)
df

,job_link,job_name,label,company_name,post_time,city_name,country_name,num_applicant,about_company,job_responsibilities,job_requirements,Seniority level,Employment type,Job function,Industries
0,https://eg.linkedin.com/jobs/view/junior-front...,Junior Frontend Developer (Remote Internship -...,web development,TalentKompass Deutschland,1 week ago,"Luxor, Luxor",Egypt,178 applicants,LOCATION:,"Assist in the development, testing, and mainte...",Basic understanding of web development princip...,Internship,Internship,Engineering and Information Technology,Human Resources Services
1,https://eg.linkedin.com/jobs/view/junior-front...,Junior Frontend Developer (Remote Internship -...,web development,TalentKompass Deutschland,1 week ago,"Cairo, Cairo",Egypt,NaN,LOCATION:,"Assist in the development, testing, and mainte...",Basic understanding of web development princip...,Internship,Internship,Engineering and Information Technology,Human Resources Services
2,https://eg.linkedin.com/jobs/view/junior-front...,Junior Frontend Developer (Remote Internship -...,web development,TalentKompass Deutschland,1 week ago,"Asyūţ, Asyut",Egypt,166 applicants,LOCATION:,"Assist in the development, testing, and mainte...",Basic understanding of web development princip...,Internship,Internship,Engineering and Information Technology,Human Resources Services
3,https://eg.linkedin.com/jobs/view/junior-fresh...,Junior/Fresh-graduate SW Support Engineer,web development,Bosta,7 months ago,"Cairo, Cairo",Egypt,NaN,NaN,Provide the technical SW Engineering support t...,Bachelor’s degree in computer science or relat...,Not Applicable,Full-time,Information Technology,"Technology, Information and Internet"
4,https://eg.linkedin.com/jobs/view/full-stack-d...,Full Stack Developer,web development,Block Gemini,3 hours ago,"Giza, Al Jizah",Egypt,NaN,NaN,Seniority levelAssociateEmployment typeFull-ti...,,Associate,Full-time,Information Technology,"Technology, Information and Internet"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,https://eg.linkedin.com/jobs/view/lead-staff-f...,Lead / Staff Full Stack Software Engineer (Ban...,web development,Agoda,2 weeks ago,"Giza, Al Jizah",Egypt,125 applicants,About Agoda,Maintain ownership and responsibility of missi...,7 years of experience developing web applicati...,Mid-Senior level,Full-time,Engineering and Information Technology,"Technology, Information and Internet"
615,https://eg.linkedin.com/jobs/view/principal-is...,Principal IS BP Engineer,web development,Valeo,1 week ago,"Cairo, Cairo",Egypt,58 applicants,Work Complexity,Coordinates and works on multiple cross-functi...,Participates in short- and long term planning ...,Mid-Senior level,Full-time,Engineering and Information Technology,Motor Vehicle Parts Manufacturing
616,https://eg.linkedin.com/jobs/view/lead-staff-s...,Lead / Staff Software Engineer Back End (Bangk...,web development,Agoda,2 weeks ago,"Alexandria, Alexandria",Egypt,73 applicants,About Agoda,Think and own the full life cycle of our produ...,7+ years’ experience under your belt developin...,Director,Full-time,Engineering and Information Technology,"Technology, Information and Internet"
617,https://eg.linkedin.com/jobs/view/senior-c%2B%...,"Senior C++ Engineer, GFI Software (Remote) - $...",web development,Crossover,6 days ago,"Cairo, Cairo",Egypt,NaN,What You Will Be Doing,Organizing and ensuring the quality and timely...,"Micro-managing - at GFI, you won't be bogged d...",Mid-Senior level,Full-time,Engineering,Software Development and IT Services and IT Co...


In [11]:
df.isna().sum()

job_link                  0
job_name                  0
label                     0
company_name              0
post_time                 0
city_name                 0
country_name              0
num_applicant           330
about_company           210
job_responsibilities      0
job_requirements          0
Seniority level           0
Employment type           0
Job function              9
Industries                7
dtype: int64

In [12]:
def save(path, df=df, save_as_csv=True, data=listings):
    import pandas as pd
    import joblib


    joblib.dump(listings, path) # load data form PC
    
    if save_as_csv:
        df.to_csv(path, index=False)

In [13]:
file_name = "linked_in_" + input_search
path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.pkl"

save(path = path)

In [14]:
df = pd.DataFrame.from_records(listings)
df.to_csv(path, index=False)

In [15]:
csv_path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.csv"

df.to_csv(csv_path, index=False)

In [16]:
listings

[{'job_link': 'https://eg.linkedin.com/jobs/view/junior-frontend-developer-remote-internship-it-at-talentkompass-deutschland-3786035181?refId=IkVTgu2qIaFaJ4yxNLFMcw%3D%3D&trackingId=14zZZoxe2QuAEuYvNfyh6w%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
  'job_name': 'Junior Frontend Developer (Remote Internship - IT)',
  'label': 'web development',
  'company_name': 'TalentKompass Deutschland',
  'post_time': '1 week ago',
  'city_name': 'Luxor, Luxor',
  'country_name': 'Egypt',
  'num_applicant': '178 applicants',
  'about_company': 'LOCATION:',
  'job_responsibilities': 'Assist in the development, testing, and maintenance of web applications using HTML, CSS, and JavaScriptCollaborate with cross-functional teams to gather requirements and design user interfacesDebug and troubleshoot frontend issues, ensuring optimal performance and user experienceImplement responsive web design principles to ensure applications render well on various devices and screen sizesAdhe

In [17]:
path

'F:\\NLP Apple Course\\NLP Project\\linked_in_web development.pkl'